In [1]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
from astropy.io import fits

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs
from mejiro.lenses import lens_util

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-065 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-067
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


In [2]:
survey_params = util.hydra_to_dict(config.survey)
snr_band = survey_params['snr_band']

pipeline_params = util.hydra_to_dict(config.pipeline)
# debugging = pipeline_params['debugging']
debugging = False

if debugging:
    pipeline_dir = f'{config.machine.pipeline_dir}_dev'
else:
    pipeline_dir = config.machine.pipeline_dir

print(f'pipeline_dir: {pipeline_dir}')

pipeline_dir: /data/bwedig/mejiro/pipeline


In [3]:
all_lenses = lens_util.get_detectable_lenses(pipeline_dir, with_subhalos=True, verbose=True, limit=10, exposure=True)

100%|██████████| 10/10 [00:00<00:00, 16.69it/s]


In [4]:
import random

lens = random.choice(all_lenses)
uid = lens.uid
print(f'uid: {uid}')

lenses = [lens]

uid: 00001074


In [5]:
arrays = []
glob_path = f'{pipeline_dir}/03/**/array_{str(uid).zfill(8)}_*.npy'
# print(glob_path)
array_paths = sorted(glob(glob_path))
pprint(array_paths)

# assert len(array_paths) == len(pipeline_params['bands']), 'Could not find an array for each band'

for array_path in array_paths:
    arrays.append(np.load(array_path))

['/data/bwedig/mejiro/pipeline/03/sca02/array_00001074_F087.npy',
 '/data/bwedig/mejiro/pipeline/03/sca02/array_00001074_F106.npy',
 '/data/bwedig/mejiro/pipeline/03/sca02/array_00001074_F129.npy',
 '/data/bwedig/mejiro/pipeline/03/sca02/array_00001074_F158.npy',
 '/data/bwedig/mejiro/pipeline/03/sca02/array_00001074_F184.npy']


In [6]:
images = []
glob_path = f'{pipeline_dir}/04/**/galsim_{str(uid).zfill(8)}_*.npy'
# print(glob_path)
image_paths = sorted(glob(glob_path))
pprint(image_paths)

# assert len(image_paths) == len(pipeline_params['bands']), 'Could not find an image for each band'

for image_path in image_paths:
    images.append(np.load(image_path))

['/data/bwedig/mejiro/pipeline/04/sca02/galsim_00001074_F087.npy',
 '/data/bwedig/mejiro/pipeline/04/sca02/galsim_00001074_F106.npy',
 '/data/bwedig/mejiro/pipeline/04/sca02/galsim_00001074_F129.npy',
 '/data/bwedig/mejiro/pipeline/04/sca02/galsim_00001074_F158.npy',
 '/data/bwedig/mejiro/pipeline/04/sca02/galsim_00001074_F184.npy']


In [7]:
# TODO TEMP: remove first element from arrays and images lists
arrays = arrays[1:]
images = images[1:]

In [8]:
pprint(lens.source_mags)
pprint(lens.lens_mags)
print(lens.detector)
print(lens.detector_position)

{'F087': 24.5239989586857,
 'F106': 24.14928357590298,
 'F129': 23.837739018910458,
 'F158': 23.588616432738927,
 'F184': 23.378862189378015}
{'F087': 20.92056622192455,
 'F106': 20.580147413398112,
 'F129': 20.251291335635383,
 'F158': 20.005874146337828,
 'F184': 19.908700137064915}
2
(1226, 409)


In [9]:
import h5py
import mejiro
import getpass
import platform
from datetime import datetime

filepath = f'{pipeline_dir}/roman_hlwas_v_0_0_2.h5'
if os.path.exists(filepath):
    os.remove(filepath)
f = h5py.File(filepath, 'a')  # append mode: read/write if exists, create otherwise

# file-level attributes
f.attrs['mejiro_version'] = (mejiro.__version__, 'mejiro version')
f.attrs['author'] = (f'{getpass.getuser()}@{platform.node()}', 'username@host for calculation')
now_string = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
f.attrs['created'] = (now_string)

In [10]:
import galsim
import lenstronomy

# create images dataset
group_images = f.create_group('images')

# set group-level attributes
group_images.attrs['lenstronomy_version'] = (lenstronomy.__version__, 'lenstronomy version')
group_images.attrs['galsim_version'] = (galsim.__version__, 'galsim version')
group_images.attrs['pixel_units'] = ('Counts/sec', 'Units of pixel values')

for lens in lenses:
    # create group for StrongLens
    group_lens = group_images.create_group(f'strong_lens_{str(uid).zfill(8)}')

    # set group-level attributes
    group_lens.attrs['uid'] = (uid, 'Unique identifier for system assigned by mejiro')
    group_lens.attrs['z_source'] = (str(lens.z_source), 'Source galaxy redshift')
    group_lens.attrs['z_lens'] = (str(lens.z_lens), 'Lens galaxy redshift')
    group_lens.attrs['d_s'] = (str(lens.d_s), 'Comoving distance to source galaxy [Gpc]')
    group_lens.attrs['d_l'] = (str(lens.d_l), 'Comoving distance to lens galaxy [Gpc]')
    group_lens.attrs['d_ls'] = (str(lens.d_ls), 'Comoving distance between lens and source [Gpc]')
    group_lens.attrs['main_halo_mass'] = (str(lens.main_halo_mass), 'Lens galaxy main halo mass [M_sun]')
    group_lens.attrs['theta_e'] = (str(lens.get_einstein_radius()), 'Einstein radius [arcsec]')
    group_lens.attrs['sigma_v'] = (str(lens.lens_vel_disp), 'Lens galaxy velocity dispersion [km/s]')
    group_lens.attrs['mu'] = (str(lens.magnification), 'Flux-weighted magnification of source')
    group_lens.attrs['num_images'] = (str(lens.num_images), 'Number of images formed by the system')
    group_lens.attrs['snr'] = (str(lens.snr), f'Signal-to-noise ratio in {snr_band} band')
    group_lens.attrs['instrument'] = ('WFI', 'Instrument')
    group_lens.attrs['exposure_time'] = (str(pipeline_params['exposure_time']), 'Exposure time [seconds]')
    # group_lens.attrs['supersampling_factor'] = (pipeline_params['grid_oversample'], 'Supersampling factor used in calculation')
    group_lens.attrs['detector'] = (str(lens.detector), 'Detector')
    group_lens.attrs['detector_position_x'] = (str(lens.detector_position[0]), 'Detector X position')
    group_lens.attrs['detector_position_y'] = (str(lens.detector_position[1]), 'Detector Y position')
    group_lens.attrs['log_mlow'] = (str(pipeline_params['log_mlow']), 'Lower mass limit for subhalos [log10(M_sun)]')
    group_lens.attrs['log_mhigh'] = (str(pipeline_params['log_mhigh']), 'Upper mass limit for subhalos [log10(M_sun)]')
    group_lens.attrs['r_tidal'] = (str(pipeline_params['r_tidal']), 'See pyHalo documentation')
    group_lens.attrs['sigma_sub'] = (str(pipeline_params['sigma_sub']), 'See pyHalo documentation')
    group_lens.attrs['num_subhalos'] = (str(lens.num_subhalos), 'Number of subhalos')

    for i, band in enumerate(pipeline_params['bands']):
        array = arrays[i]
        image = images[i]

        dataset_synth = group_lens.create_dataset(f'synthetic_image_{str(uid).zfill(8)}_{band}', data=array)
        dataset_exposure = group_lens.create_dataset(f'exposure_{str(uid).zfill(8)}_{band}', data=image)

        # set synthetic image dataset attributes
        dataset_synth.attrs['pixel_scale'] = (str(0.11 / pipeline_params['grid_oversample']), 'Pixel scale [arcsec/pixel]')
        dataset_synth.attrs['fov'] = (str(pipeline_params['side']), 'Field of view [arcsec]')
        
        # set exposure dataset attributes
        dataset_exposure.attrs['pixel_scale'] = (str(0.11), 'Pixel scale [arcsec/pixel]')
        dataset_exposure.attrs['fov'] = (str(0.11 * pipeline_params['final_pixel_side']), 'Field of view [arcsec]')

        # attributes to set on both
        for dset in [dataset_synth, dataset_exposure]:
            dset.attrs['filter'] = (band, 'Filter')
            dset.attrs['source_magnitude'] = (str(lens.source_mags[band]), 'Unlensed source galaxy AB magnitude')
            dset.attrs['lensed_source_magnitude'] = (str(lens.lensed_source_mags[band]), 'Lensed source galaxy AB magnitude')
            dset.attrs['lens_magnitude'] = (str(lens.lens_mags[band]), 'Lens galaxy AB magnitude')


In [11]:
import webbpsf
from mejiro.utils import roman_util

# set detectors and detector_positions
detectors = range(1, 19)
detector_positions = roman_util.divide_up_sca(5)

# create psfs dataset
group_psfs = f.create_group('psfs')

# set group-level attributes
group_images.attrs['webbpsf_version'] = (webbpsf.__version__, 'WebbPSF version')

for det in detectors:
    # create group for detector
    group_detector = group_psfs.create_group(f'SCA{str(det).zfill(2)}')

    for det_pos in detector_positions:
        for i, band in enumerate(pipeline_params['bands']):
            psf_image = util.unpickle(f'/data/bwedig/mejiro/cached_psfs/{band}_{det}_{det_pos[0]}_{det_pos[1]}_5_101.pkl')
            psf = psf_image.image.array

            dataset_psf = group_detector.create_dataset(f'psf_{det}_{det_pos[0]}_{det_pos[1]}_{band}', data=psf)

            # set psf dataset attributes
            dataset_psf.attrs['detector'] = (str(det), 'Detector')
            dataset_psf.attrs['detector_position_x'] = (str(det_pos[0]), 'Detector X position')
            dataset_psf.attrs['detector_position_y'] = (str(det_pos[1]), 'Detector Y position')
            dataset_psf.attrs['fov_pixels'] = (str(101), 'See WebbPSF documentation')
            dataset_psf.attrs['oversample'] = (str(pipeline_params['grid_oversample']), 'See WebbPSF documentation')

In [12]:
def h5_tree(val, pre=''):
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                print(pre + '└── ' + key)
                h5_tree(val, pre+'    ')
            else:
                try:
                    print(pre + '└── ' + key + ' (%d)' % len(val))
                except TypeError:
                    print(pre + '└── ' + key + ' (scalar)')
        else:
            if type(val) == h5py._hl.group.Group:
                print(pre + '├── ' + key)
                h5_tree(val, pre+'│   ')
            else:
                try:
                    print(pre + '├── ' + key + ' (%d)' % len(val))
                except TypeError:
                    print(pre + '├── ' + key + ' (scalar)')

In [13]:
with h5py.File(filepath, 'r') as hf:
    print(hf)
    h5_tree(hf)

<HDF5 file "roman_hlwas_v_0_0_2.h5" (mode r+)>
├── images
│   └── strong_lens_00001074
│       ├── exposure_00001074_F106 (91)
│       ├── exposure_00001074_F129 (91)
│       ├── exposure_00001074_F158 (91)
│       ├── exposure_00001074_F184 (91)
│       ├── synthetic_image_00001074_F106 (485)
│       ├── synthetic_image_00001074_F129 (485)
│       ├── synthetic_image_00001074_F158 (485)
│       └── synthetic_image_00001074_F184 (485)
└── psfs
    ├── SCA01
    │   ├── psf_1_1226_1226_F106 (505)
    │   ├── psf_1_1226_1226_F129 (505)
    │   ├── psf_1_1226_1226_F158 (505)
    │   ├── psf_1_1226_1226_F184 (505)
    │   ├── psf_1_1226_2044_F106 (505)
    │   ├── psf_1_1226_2044_F129 (505)
    │   ├── psf_1_1226_2044_F158 (505)
    │   ├── psf_1_1226_2044_F184 (505)
    │   ├── psf_1_1226_2862_F106 (505)
    │   ├── psf_1_1226_2862_F129 (505)
    │   ├── psf_1_1226_2862_F158 (505)
    │   ├── psf_1_1226_2862_F184 (505)
    │   ├── psf_1_1226_3679_F106 (505)
    │   ├── psf_1_1226_3679_F12

In [ ]:
with h5py.File(filepath, 'r') as hf:
    images_group = hf['images']
    strong_lens_group = images_group['strong_lens_00003163']
    
    for k in strong_lens_group.attrs.keys():
        print('{} => {}'.format(k, strong_lens_group.attrs[k]))

    print('\n')

    exposure_f106 = strong_lens_group['exposure_00003163_F106']
    for k in exposure_f106.attrs.keys():
        print('{} => {}'.format(k, exposure_f106.attrs[k]))